In [47]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import pandas as pd
import time
from W_settings import *
import datetime
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import os


In [38]:
separation_date = '31/12/2010'
begin_test = '01/01/2015'
test_date = '29/11/2014'
begin_date = '01/01/2005'
final_date = '31/12/2015'

In [39]:
def get_season(dt):
    # it returnts string
    #print('type of dt:' + str(type(dt)))
    #print(dt)
    dt = dt.date()
    if (dt.month < 3):
        return 'H'
    elif (dt.month < 6):
        return 'P'
    elif (dt.month < 9):
        return "E"
    elif (dt.month < 12):
        return "A"
    else:
        return "H"


In [40]:
def prepare_data(df_temp, keep_season=False, keep_wind=False):

    train_data = df_temp[df_temp['Date'] < test_date].drop(["Date"], axis=1).dropna(axis=0, how="any")
    test_data = df_temp[df_temp['Date'] > test_date].drop(["Date"], axis=1).dropna(axis=0, how="any")

    season_train = train_data['Season']
    season_test = test_data['Season']

    train_data = train_data.drop("Season", axis=1)
    test_data = test_data.drop("Season", axis=1)

    vent_train = train_data["vent"]
    vent_test = test_data["vent"]

    train_data = train_data.drop("vent", axis=1)
    test_data = test_data.drop("vent", axis=1)

    """
    for col in salouel_data.columns:
        train_data[col] = train_data[col].apply(scale_transformer, args=(settings_array[col],))

    for col in test_data.columns:
        test_data[col] = test_data[col].apply(scale_transformer, args=(settings_array[col],))
    """
    if(keep_season):
        train_data["Season"] = season_train
        test_data["Season"] = season_test

    if(keep_wind):
        train_data["vent"] = vent_train
        test_data["vent"] = vent_test

    train_data = pd.get_dummies(train_data)
    test_data = pd.get_dummies(test_data)
    #print(test_data.info())
    train_data.to_csv('./data/train.csv')

    test_data.to_csv('./data/test.csv')

    print("Dataframes saved...")
    return train_data, test_data



In [41]:
cols = ['Date', 'NO2', 'O3', 'PM10', 'RR', 'TN', 'TX', 'TM',
                     'PMERM', 'FFM', 'DXY', 'UM', 'GLOT']


In [42]:

def preprocessing_data(df):
    df = df.iloc[1:]
    df.columns = cols
    df = df.drop(["Date", 'NO2', 'O3'], axis=1)
    df = df.astype(float)
    df['vent'] = df['DXY'].apply(get_dir)
    df = df.drop(['DXY'], axis=1)
    #df = df.dropna(axis=0, how="any")
    return df


In [43]:
from pandas import read_csv
creil_raw_data = pd.read_csv("Moyennes_J_creil_2005_2015.csv", header=None, sep=';', decimal=',')
roth_raw_data = pd.read_csv("Moyennes_J_roth_2005_2015.csv", header=None, sep=';', decimal=',')
salouel_raw_data = pd.read_csv("Moyennes_J_salouel_2005_2015.csv", header=None, sep=';', decimal=',')


In [44]:
def clean_data(array_raw_data, cut):
    global begin_date
    # concat is first
    conc = True
    df = array_raw_data.iloc[1:]
    df.columns = cols
    df_temp = df.drop(["Date", 'NO2', 'O3'], axis=1)
    df_temp = df_temp.astype(float)
    df_temp["Date"] = pd.to_datetime(df["Date"])
    if cut == True:
        df_temp = df_temp[df_temp['Date'] > separation_date]
        begin_date = '01/01/2010'
    else:
        begin_date = '01/01/2005'
    # df_temp["Date"] = pd.to_datetime(df["Date"])
    df_temp['Season'] = df_temp["Date"].apply(get_season)
    df_temp['vent'] = df_temp['DXY'].apply(get_dir)

    return df_temp

In [61]:

df_temp = clean_data(salouel_raw_data, False)

train_data, test_data = prepare_data(df_temp, True, True)
X_train = train_data.loc[:, [ 'TN', 'RR','TX', 'TM', 'PMERM', 'FFM', 'UM', 'GLOT']].values.copy()
X_test = test_data.loc[:, ['PM10', 'RR', 'TN', 'TX', 'TM', 'PMERM', 'FFM', 'UM', 'GLOT']].values.copy()

X_std = StandardScaler().fit_transform(X_train);  # standardization of data

# Fit the model with X_std and apply the dimensionality reduction on X_std.
pca = PCA()

Dataframes saved...


In [62]:
Y_pca = pca.fit_transform(X_std)


In [63]:

pca.explained_variance_ratio_

array([0.45998931, 0.21455993, 0.13755863, 0.08423133, 0.07358076,
       0.01992791, 0.00930092, 0.0008512 ])

array([0.45998931, 0.21455993, 0.13755863, 0.08423133, 0.07358076,
       0.01992791, 0.00930092, 0.0008512 ])